In [1]:
# import section
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler, MinMaxScaler, PowerTransformer, OrdinalEncoder
from sklearn.linear_model import Lasso, Ridge, SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

# Read data:

In [3]:
ds = pd.read_csv("data/diamonds.csv", index_col = "Unnamed: 0")

**First look at the data:**

In [5]:
print("______________________________________________________________________________")
print("Number of columns in the dataset:", len(ds.columns))
print("Number of rows in the dataset:", ds.shape[0])
print(ds.head())
print("______________________________________________________________________________")
print(ds.describe())
print("______________________________________________________________________________")
print(ds.info())
print("______________________________________________________________________________")

______________________________________________________________________________
Number of columns in the dataset: 10
Number of rows in the dataset: 53940
   carat      cut color clarity  depth  table  price     x     y     z
1   0.23    Ideal     E     SI2   61.5   55.0    326  3.95  3.98  2.43
2   0.21  Premium     E     SI1   59.8   61.0    326  3.89  3.84  2.31
3   0.23     Good     E     VS1   56.9   65.0    327  4.05  4.07  2.31
4   0.29  Premium     I     VS2   62.4   58.0    334  4.20  4.23  2.63
5   0.31     Good     J     SI2   63.3   58.0    335  4.34  4.35  2.75
______________________________________________________________________________
              carat         depth         table         price             x  \
count  53940.000000  53940.000000  53940.000000  53940.000000  53940.000000   
mean       0.797940     61.749405     57.457184   3932.799722      5.731157   
std        0.474011      1.432621      2.234491   3989.439738      1.121761   
min        0.200000     43

After first look at the dataset we can conclude that the dataset have no missing values. DS contain 10 columns totally (3 categorical columns, 7 numerical).

Let's split the dataset into train and test sets. Test set we are gonna use for final predictions. For validation of our model we'll use cross-validation technique.

In [6]:
train, test = train_test_split(ds, test_size = 0.3, random_state = 42)
print("______________________________________________________________________________")
print("Number of rows in the train dataset:", train.shape[0])
print(train.head())
print("______________________________________________________________________________")
print("Number of rows in the train dataset:", test.shape[0])
print(test.head())

______________________________________________________________________________
Number of rows in the train dataset: 37758
       carat    cut color clarity  depth  table  price     x     y     z
19498   1.21  Ideal     H    VVS2   61.3   57.0   8131  6.92  6.87  4.23
31230   0.31  Ideal     E     VS2   62.0   56.0    756  4.38  4.36  2.71
22312   1.21  Ideal     E     VS1   62.4   57.0  10351  6.75  6.83  4.24
279     0.81  Ideal     F     SI2   62.6   55.0   2795  5.92  5.96  3.72
6647    0.79  Ideal     I    VVS2   61.7   56.0   4092  5.94  5.95  3.67
______________________________________________________________________________
Number of rows in the train dataset: 16182
       carat        cut color clarity  depth  table  price     x     y     z
1389    0.24      Ideal     G    VVS1   62.1   56.0    559  3.97  4.00  2.47
50053   0.58  Very Good     F    VVS2   60.0   57.0   2201  5.44  5.42  3.26
41646   0.40      Ideal     E    VVS2   62.1   55.0   1238  4.76  4.74  2.95
42378   0.

In [7]:
train.to_csv(path_or_buf='data/train.csv', index=False)
test.to_csv(path_or_buf='data/test.csv', index=False)

# Let's provide some EDA on this dataset

In [ ]:
# At first let's look on target - "price" column:
sns.distplot(a=train['price'], kde=True)

"Price" column data are not normally distributed. Most of the data distributed at low prices. Maybe transformation distribution to normal will increase performance of the model.

In [ ]:
# Let's look at Pearson correlation matrix
cor_mat = train.corr()
plt.figure(figsize=(12,8))
sns.heatmap(cor_mat, annot=True, cmap=plt.cm.Reds)

As we can see a lot of columns are very correlates within each other. So maybe we'll need to drop some of the columns in order to prevent multicollinearity. We'll check this during model construction.

Now let's get some plots for categorical columns:

In [ ]:
# Obtain lists of cat and num columns
num_features = [col for col in train.columns if train[col].dtype in ['int64', 'float64']] # numeric features
cat_features = [col for col in train.columns if train[col].dtype == 'object'] # categorical features
num_features.remove('price')

print("Categorical features:", cat_features)
print("Numerical features:", num_features)

# Let's look on categorical features:
print("Num of cut column unique values:", train.cut.unique())
print("Num of color column unique values:", train.color.unique())
print("Num of clarity column unique values:", train.clarity.unique())   

# "cut" column
plt.figure(figsize=(15,6))
fig = plt.figure(figsize=(14,4))
fig.subplots_adjust(hspace=0.3, wspace=0.3)
ax = fig.add_subplot(1, 2, 1)
sns.countplot(x=train.cut).set_title("Cut values dictribution")
ax = fig.add_subplot(1, 2, 2)
sns.boxplot(x=train.cut, y=train.price).set_title("Cut vs price boxplot")

# "color" column
plt.figure(figsize=(15,6))
fig = plt.figure(figsize=(14,4))
fig.subplots_adjust(hspace=0.3, wspace=0.3)
ax = fig.add_subplot(1, 2, 1)
sns.countplot(x=train.color).set_title("Color values dictribution")
ax = fig.add_subplot(1, 2, 2)
sns.boxplot(x=train.color, y=train.price).set_title("Color vs price boxplot")

# "clarity" column
plt.figure(figsize=(15,6))
fig = plt.figure(figsize=(14,4))
fig.subplots_adjust(hspace=0.3, wspace=0.3)
ax = fig.add_subplot(1, 2, 1)
sns.countplot(x=train.clarity).set_title("Clarity values dictribution")
ax = fig.add_subplot(1, 2, 2)
sns.boxplot(x=train.clarity, y=train.price).set_title("Clarity vs price boxplot")

Now let's look on numerical features:

In [ ]:
fig = plt.figure(figsize=(16,12))
fig.subplots_adjust(hspace=0.4, wspace=0.4)

# we'll iterate through all numerical columns in train and build regplot ("price" dependent) for every column: 
for i in range(1, len(num_features)+1):
    ax = fig.add_subplot(2, 3, i)
    sns.regplot(x=train[num_features[i-1]], y=train['price'], ax=ax).set_title("Price vs "+str(num_features[i-1]))

Features such as "x", "y", "z" and "carat"  have visible impact on price. On the contrary table and depth features seems not to have great impact on price.
From regplots several outliers can be detected. As we can see there are several observations with x,y,z values which is impossible. So either these data are missing or some mistakes done in dataset. Since such observations not much we'll drop them from both datasets (train and test). Also we can see couple of outliers with y>30 and z>30. We'll drop these observations as well.

In [ ]:
# drop outliers
train = train.drop(train[train["x"]==0].index)
train = train.drop(train[train["y"]==0].index)
train = train.drop(train[train["z"]==0].index)
train = train.drop(train[train["y"]>30].index)
train = train.drop(train[train["z"]>30].index)
test = test.drop(test[test["x"]==0].index)
test = test.drop(test[test["y"]==0].index)
test = test.drop(test[test["z"]==0].index)

Let's check again on numercial features after removing outliers:

In [ ]:
fig = plt.figure(figsize=(16,12))
fig.subplots_adjust(hspace=0.2, wspace=0.2)

# we'll iterate through all numerical columns in train and build regplot ("price" dependent) for every column: 
for i in range(1, len(num_features)+1):
    ax = fig.add_subplot(2, 3, i)
    sns.regplot(x=train[num_features[i-1]], y=train['price'], ax=ax).set_title("Price vs "+str(num_features[i-1]))

After removing outliers data look a bit cleaner. Still it can be seen couple of outliers, however we'll leave them as it is, beacause they might be representative for modeling.

Finally let's look on numerical features dictribution:

In [ ]:
fig = plt.figure(figsize=(15,10))
fig.subplots_adjust(hspace=0.2, wspace=0.2)

# we'll iterate through all numerical columns in train and build regplot ("price" dependent) for every column: 

plot_list = ["x", "y", "z", "carat"]
for i in range(1, len(plot_list)+1):
    ax = fig.add_subplot(2, 2, i)
    sns.kdeplot(x=train[plot_list[i-1]], ax=ax).set_title("KDE plot for "+str(plot_list[i-1]))

As result of EDA we can conclude:
- numerical features such as "x", "y", "z" and "carat" have a great impact on diamond price. However these features are also dependent among themselves, so maybe we need to remove some of these features or create new features (something like x+y+z). We'll check this during the further modeling process. Probably, also some scaling for these features needed. We'll also check several options during modeling process.
- numerical features such as "table" and "depth" seems to not have big influence on diamond price. Thus, maybe we should delete them as well.
- all three categorical features ("cut", "color", "clarity") can be important for diamond price prediction, however they are unclear. In each group of these features there are available diamonds with big prices as well as diamonds with low prices. So we'll leave these features in dataset. From boxplots for these features it's unclear how we should encode these features. On one side it have to be clear order in them (for example: "cut" - quality of the cut (Fair, Good, Very Good, Premium, Ideal)), on the other side its not seen on plots such order. So we will try two options: label encoding and one-hot encoding.

# Preprocessing of data:

Let's try two approaches with categorical features encoding: Ordinal Encoding and One-Hot Encoding. Since all categorical features supposed to have some order in their values Ordinal Encoding should provide better results, However, we'll check if this correct for our dataset.

In [ ]:
# In order to choose best preprocessing approaches we'll create a function called bunch_of_models 
# that creates a bunch of simple models and print their performance on passed dataset. Beside this we'll create validation dataset from train set.

# Assign X_train, y_train, X_test, y_test
X_train = train.drop(['price'], axis=1)
y_train = train["price"]
X_test = test.drop(['price'], axis=1)
y_test = test["price"]

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.3, random_state = 42)

def bunch_of_models(X_train, y_train, X_valid, y_valid, model_dict):
    for model_name, model in model_dict.items():
        if model_name == 'XGB':
            model.fit(X_train, y_train, early_stopping_rounds=10, eval_set=[(X_valid, y_valid)])
            pred = model.predict(X_valid)
            feat_imp = model.feature_importances_
            feat_imp_df = pd.DataFrame(data=feat_imp, index=X_train.columns, columns=['Feature_importance'])
            plt.figure(figsize=(10,6))
            sns.barplot(x=feat_imp_df.index, y=feat_imp_df.Feature_importance).set_title(str(model_name) + " feature importance:")
        else:
            model.fit(X_train, y_train)
            pred = model.predict(X_valid)
        print("___________________________________")
        print(model_name + " MAE:", mean_absolute_error(pred, y_valid))
        # print(model_name + " f1_score:", f1_score(pred, y_test))
        # print(model_name + " confusion matrix:", confusion_matrix(pred, y_test))
        print("___________________________________")
        


In [ ]:
# Ordinal encoding of 'cut', 'color' and 'clarity' features
X_train_ord_enc = X_train.copy()
X_valid_ord_enc = X_valid.copy()
X_test_ord_enc = X_test.copy()
X_train_ord_enc['cut'] = X_train_ord_enc['cut'].map({'Fair':1, 'Good':2, 'Very Good':3, 'Premium':4, 'Ideal':5}).astype(int)
X_train_ord_enc['color'] = X_train_ord_enc['color'].map({'J':1, 'I':2, 'H':3, 'G':4, 'F':5, 'E':6, 'D':7}).astype(int)
X_train_ord_enc['clarity'] = X_train_ord_enc['clarity'].map({'I1':1, 'SI2':2, 'SI1':3, 'VS2':4, 'VS1':5, 'VVS2':6, 'VVS1':7, 'IF':8}).astype(int)
X_valid_ord_enc['cut'] = X_valid_ord_enc['cut'].map({'Fair':1, 'Good':2, 'Very Good':3, 'Premium':4, 'Ideal':5}).astype(int)
X_valid_ord_enc['color'] = X_valid_ord_enc['color'].map({'J':1, 'I':2, 'H':3, 'G':4, 'F':5, 'E':6, 'D':7}).astype(int)
X_valid_ord_enc['clarity'] = X_valid_ord_enc['clarity'].map({'I1':1, 'SI2':2, 'SI1':3, 'VS2':4, 'VS1':5, 'VVS2':6, 'VVS1':7, 'IF':8}).astype(int)
X_test_ord_enc['cut'] = X_test_ord_enc['cut'].map({'Fair':1, 'Good':2, 'Very Good':3, 'Premium':4, 'Ideal':5}).astype(int)
X_test_ord_enc['color'] = X_test_ord_enc['color'].map({'J':1, 'I':2, 'H':3, 'G':4, 'F':5, 'E':6, 'D':7}).astype(int)
X_test_ord_enc['clarity'] = X_test_ord_enc['clarity'].map({'I1':1, 'SI2':2, 'SI1':3, 'VS2':4, 'VS1':5, 'VVS2':6, 'VVS1':7, 'IF':8}).astype(int)

print("After Ordinal Encoding:\n", X_train_ord_enc.loc[:, ['cut', 'color', 'clarity']])
print("After Ordinal Encoding:\n", X_valid_ord_enc.loc[:, ['cut', 'color', 'clarity']])
print("After Ordinal Encoding:\n", X_test_ord_enc.loc[:, ['cut', 'color', 'clarity']])

# One-Hot Encoding of 'cut', 'color' and 'clarity' features
X_train_one_hot = X_train.copy()
X_valid_one_hot = X_valid.copy()

X_train_one_hot = pd.get_dummies(X_train_one_hot)
X_valid_one_hot = pd.get_dummies(X_valid_one_hot)

In [ ]:
# Now let's check what method performs better during modeling. 
# We'll pass to bunch_of_models func several simple regressor models.

model_dict = dict([
    ('Lasso', Lasso()),
    ('Ridge', Ridge()),
    ('SGD', SGDRegressor()),
    ('SVM', SVR()),
    ('AdaBoost', AdaBoostRegressor()),
    ('RF', RandomForestRegressor()),
    ('XGB', XGBRegressor())
])

print("Ordinal encoding performance:\n")
bunch_of_models(X_train_ord_enc, y_train, X_valid_ord_enc, y_valid, model_dict)
print("One-Hot Encoding performance:\n")
bunch_of_models(X_train_one_hot, y_train, X_valid_one_hot, y_valid, model_dict)

As we can see RandomForest model preforms much better then any other. Compared two encoding approaches One-Hot Encoding works better for linear models, however for other (especially for RF model) ordinal encoding works better. SGD model show something weird, probably data need to be additionally preprocessed for this model or model have to be tuned.

Let's check feature importances in ordinal encoded dataset using RandomForrest and XGBoost models:

In [ ]:
rf_model = RandomForestRegressor()
rf_model.fit(X_train_ord_enc, y_train)
rf_pred = rf_model.predict(X_valid_ord_enc)
print("RF MAE:", mean_absolute_error(rf_pred, y_valid))

xgb_model = XGBRegressor()
xgb_model.fit(X_train_ord_enc, y_train, early_stopping_rounds=10, eval_set=[(X_valid_ord_enc, y_valid)])
xgb_pred = rf_model.predict(X_valid_ord_enc)
print("XGB MAE:", mean_absolute_error(xgb_pred, y_valid))

rf_feat_imp = pd.DataFrame(data=rf_model.feature_importances_, index=X_train_ord_enc.columns, columns=['Feature_importance'])
xgb_feat_imp = pd.DataFrame(data=xgb_model.feature_importances_, index=X_train_ord_enc.columns, columns=['Feature_importance'])

fig = plt.figure(figsize=(15, 6))
fig.subplots_adjust(hspace=0.2, wspace=0.2)
ax = fig.add_subplot(1, 2, 1)
sns.barplot(x=rf_feat_imp.index, y=rf_feat_imp.Feature_importance).set_title("Random Forest feature importance:")
ax = fig.add_subplot(1, 2, 2)
sns.barplot(x=xgb_feat_imp.index, y=xgb_feat_imp.Feature_importance).set_title("XGBoost feature importance:")

It seems that 'depth' and 'table' columns doesn't have any impact on both RF and XGB models. 

Now let's try to drop a couple of columns and check whether it change performance sufficiently.

In [ ]:
# drop 'depth' and 'table' columns
X_train_ord_enc_red = X_train_ord_enc.drop(['depth', 'table'], axis=1)
X_valid_ord_enc_red = X_valid_ord_enc.drop(['depth', 'table'], axis=1)

X_train_one_hot_red = X_train_one_hot.drop(['depth', 'table'], axis=1)
X_valid_one_hot_red = X_valid_one_hot.drop(['depth', 'table'], axis=1)

# and check again performance
print("Ordinal encoding performance:\n")
bunch_of_models(X_train_ord_enc_red, y_train, X_valid_ord_enc_red, y_valid, model_dict)
print("One-Hot Encoding performance:\n")
bunch_of_models(X_train_one_hot_red, y_train, X_valid_one_hot_red, y_valid, model_dict)

For most of the models performance is pretty much the same after droping 'depth' and 'table' columns. So for further modeling we are not gonna use these columns in order to reduce datasets size. Surprisingly, SGD model performed much better after dropping operation.

Now let's try to create several new features. We are gonna use ordinal encoded dataset.

In [ ]:
# drop 'depth' and 'table' columns
X_train_ord_enc.drop(['depth', 'table'], axis=1, inplace=True)
X_valid_ord_enc.drop(['depth', 'table'], axis=1, inplace=True)

X_train_new = X_train_ord_enc.copy()
X_valid_new = X_valid_ord_enc.copy()

# create sum of x, y, z and their squares
X_train_new["xyz_sum"] = X_train_new['x'] + X_train_new['y'] + X_train_new['z']  # sum of x, y, z
X_train_new['x^2'] = X_train_new['x'] * X_train_new['x'] # square x
X_train_new['y^2'] = X_train_new['y'] * X_train_new['y'] # square y
X_train_new['z^2'] = X_train_new['z'] * X_train_new['z'] # square z
X_train_new['carat'] = X_train_new['carat'] * X_train_new['carat'] # square carat value

X_valid_new["xyz_sum"] = X_valid_new['x'] + X_valid_new['y'] + X_valid_new['z']
X_valid_new['x^2'] = X_valid_new['x'] * X_valid_new['x']
X_valid_new['y^2'] = X_valid_new['y'] * X_valid_new['y']
X_valid_new['z^2'] = X_valid_new['z'] * X_valid_new['z']
X_valid_new['carat'] = X_valid_new['carat'] * X_valid_new['carat']

# and check again performance
print("Ordinal encoding performance:\n")
bunch_of_models(X_train_new, y_train, X_valid_new, y_valid, model_dict)


Adding other features does not improve performance of any model. So we'll not add these features into our datasets.

Now let's construct Pipeline inside GridSearch in oreder to find the best scaling and encoding approaches for dataset and tune model. We are gonna create two separate GridSearch Pipelines with RandomForest model and with XGBoost model accordingly. 

In [ ]:
# At first let's define pipeline_constructor function that constructs preprocessor for handling our data before modeling
def grid_search_pipeline(X_train, y_train, model=XGBRegressor(), param_grid=None):
    """
    Construct Pipelines for numerical and categorical columns.
    Create self.preprocessor and self.preprocessor_dict for final Pipeline and further GridSearch
    """

    # At first we'll find columns with numerical and categorical values
    num_features = [col for col in X_train.columns if X_train[col].dtype in ['int64', 'float64']]
    cat_features = [col for col in X_train.columns if X_train[col].dtype == 'object']

    # Pipeline for numerical features that contains scaling and normalization operations
    num_transformer = Pipeline(steps=[
        ('scaler', 'passthrough'),
        ('norm', 'passthrough')
    ])

    # Pipeline for categorical features that contains encoding operation
    cat_transformer = Pipeline(steps=[
        ('encode', 'passthrough')
    ])

    # Combining of num and cat Pipelines into one preprocessor step using ColumnTransformer.
    # This preprocessor will be used in final Pipeline and further in GridSearch
    preprocessor = ColumnTransformer(transformers=[
        ('num', num_transformer, num_features),
        ('cat', cat_transformer, cat_features)
    ])
    
    # Constructing of final Pipeline taht contains preprocessor and regression model
    final_estimator = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    
    # Finally GridSearch construction. As estimator parameter of GridSearch we'll pass our final Pipeline:
    # If model is XGBoost then we additionally split dataset to train and validation in order to have possibility of early_stopping using
    if model == XGBRegressor():
        X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.3, random_state = 42)
        grid_search = GridSearchCV(regressor, param_grid=param_grid, scoring='neg_mean_absolute_error', n_jobs=-1)
        grid_search.fit(X_train, y_train, early_stopping_rounds=20, eval_set=[(X_valid, y_valid)])
    else:
        grid_search = GridSearchCV(final_estimator, param_grid=param_grid, scoring='neg_mean_absolute_error', n_jobs=-1)
        grid_search.fit(X_train, y_train)

    return grid_search.best_score_, grid_search.best_estimator_, grid_search.best_params_

In [ ]:
# Let's again reassign X_train, y_train, X_test, y_test. Now X_train doesn't have any preprocessing except outliers removing and removing of 'depth' and 'table' columns.
X_train = train.drop(['price', 'depth', 'table'], axis=1)
y_train = train["price"]
X_test = test.drop(['price','depth', 'table'], axis=1)
y_test = test["price"]

In [ ]:
# So now we are gonna pass into GridSearch two models: RF and XGB.
# We'll create two parameter dictionaries for every model. We try to checck different scaling and encoding approaches, as well as different hyperparameters values for regressors.

# categories for OrdinalEncoder
cut_categories = ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']
color_categories = ['J', 'I', 'H', 'G', 'F', 'E', 'D']
clarity_categories = ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']

# dict with preprocessing parameters
preprocessor_dict = dict(preprocessor__num__scaler=[StandardScaler(), RobustScaler(), MinMaxScaler()],
                         preprocessor__num__norm=[PowerTransformer()],
                         preprocessor__cat__encode=[OrdinalEncoder(categories=[cut_categories, color_categories, clarity_categories]), OneHotEncoder()]
                        )

# dict with RandomForest model hyperparameters
RFC_dict = dict(regressor__n_estimators=[100, 500, 900],
                regressor__min_samples_leaf=[1, 2, 4])
RFC_dict.update(preprocessor_dict)

# dict with XGBoost model hyperparameters
XGB_dict = dict(regressor__n_estimators=[100, 500],
                regressor__max_depth=[1, 2])
XGB_dict.update(preprocessor_dict)

# Finally let's find best estimators based on RF and XGB
RF_bs, RF_be, RF_bp =  grid_search_pipeline(X_train, y_train, model=RandomForestRegressor(), param_grid=preprocessor_dict)
#XGB_bs, XGB_be, XGB_bp = grid_search_pipeline(X_train, y_train, model=XGBRegressor(), param_grid=XGB_dict)

Now let's display the results of GridSearch estimation:

In [ ]:
print(-RF_bs)
print(RF_bp)

Now we can make predictions of test dataset:

In [ ]:
pred = RF_be.predict(X_test)
print(" MAE:", mean_absolute_error(pred, y_test))
